In [1]:
import pandas as pd
import pandasql as ps

# Корабли
Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.
Отношение Classes содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах). В отношении Ships записаны название корабля, имя его класса и год спуска на воду. В отношение Battles включены название и дата битвы, в которой участвовали корабли, а в отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).
Замечания. 1) В отношение Outcomes могут входить корабли, отсутствующие в отношении Ships. 2) Потопленный корабль в последующих битвах участия не принимает.

![](ships.png)

In [2]:
classes = pd.read_csv('data/classes.csv', index_col='class')
classes.head()

,type,country,guns,bore,displacement
class,,,,,
Bismarck,bb,Germany,8,15.0,42000
Iowa,bb,USA,9,16.0,46000
Kongo,bc,Japan,8,14.0,32000
North Carolina,bb,USA,12,16.0,37000
Renown,bc,Gt.Britain,6,15.0,32000


In [3]:
ships = pd.read_csv('data/ships.csv', index_col='name')
ships.head()

,class,launched
name,,
California,Tennessee,1921
Haruna,Kongo,1916
Hiei,Kongo,1914
Iowa,Iowa,1943
Kirishima,Kongo,1915


In [4]:
outcomes = pd.read_csv('data/outcomes.csv', index_col=['ship', 'battle'])
outcomes.head()

,,result
ship,battle,
Bismarck,North Atlantic,sunk
California,Surigao Strait,OK
Duke of York,North Cape,OK
Fuso,Surigao Strait,sunk
Hood,North Atlantic,sunk


In [5]:
battles = pd.read_csv('data/battles.csv', index_col='name', parse_dates=['date'])
battles.head()

,date
name,
Guadalcanal,1942-11-15
North Atlantic,1941-05-25
North Cape,1943-12-26
Surigao Strait,1944-10-25


# Задача 1
По Вашингтонскому международному договору от начала 1922 г. запрещалось строить линейные корабли водоизмещением более 35 тыс.тонн. Укажите корабли, нарушившие этот договор (учитывать только корабли c известным годом спуска на воду). Вывести названия кораблей.

![](ships.png)

### SQL

In [6]:
query = '''
    select 
        ships.name 
    from 
        ships 
        join 
        classes 
        on ships.class = classes.class 
    where classes.displacement > 35000 and ships.launched >= 1922 and classes.type = 'bb'
'''
ps.sqldf(query, locals())

,name
0,Iowa
1,Missouri
2,Musashi
3,New Jersey
4,North Carolina
5,Washington
6,Wisconsin
7,Yamato
8,South Dakota


### Pandas

In [7]:
ship_n_classes = ships.join(classes, 'class')
ship_n_classes[
    (ship_n_classes['displacement'] > 35000) & 
    (ship_n_classes['launched'] >= 1922) & 
    (ship_n_classes['type'] == 'bb')
].reset_index()[['name']]

,name
0,Iowa
1,Missouri
2,Musashi
3,New Jersey
4,North Carolina
5,Washington
6,Wisconsin
7,Yamato
8,South Dakota


# Задача 2

Найдите корабли, "сохранившиеся для будущих сражений"; т.е. выведенные из строя в одной битве (damaged), они участвовали в другой, произошедшей позже. Здесь важно учесть дату.

![](ships.png)

### SQL

In [8]:
query = '''
    select 
        outcomes_1.ship
    from 
        outcomes as outcomes_1
        join
        battles as battles_1
        on outcomes_1.battle = battles_1.name
        join
        outcomes as outcomes_2
        on outcomes_1.ship = outcomes_2.ship and outcomes_1.result = 'damaged'
        join
        battles as battles_2
        on outcomes_2.battle = battles_2.name and battles_1.date < battles_2.date
    '''

ps.sqldf(query, locals())

,ship
0,California


### Pandas

In [9]:
outcomes_n_battles = outcomes.reset_index().merge(battles, left_on='battle', right_on='name')
outcomes_n_battles_n_outcomes_n_battles = outcomes_n_battles[outcomes_n_battles['result'] == 'damaged'] \
                                            .merge(outcomes_n_battles, left_on='ship', right_on='ship')
outcomes_n_battles_n_outcomes_n_battles[
    outcomes_n_battles_n_outcomes_n_battles['date_x'] < outcomes_n_battles_n_outcomes_n_battles['date_y']
][['ship']]

,ship
2,California


# Задача 3
Укажите названия, водоизмещение и число орудий, кораблей участвовавших в сражении при Гвадалканале (Guadalcanal).
![](ships.png)

### SQL

In [10]:
query = '''
    select 
        outcomes.ship, 
        classes.displacement, 
        classes.guns
    from 
        outcomes
        left join
        ships
        on outcomes.ship = ships.name
        left join
        classes
        on ships.class = classes.class
    where 
        outcomes.battle = 'Guadalcanal'
    '''

ps.sqldf(query, locals())

,ship,displacement,guns
0,Kirishima,32000,8
1,South Dakota,37000,12
2,Washington,37000,12
3,California,32000,12


### Pandas

In [11]:
outcomes.reset_index()[outcomes.reset_index()['battle'] == 'Guadalcanal'] \
    .merge(ships, left_on='ship', right_on='name') \
    .join(classes, 'class') \
    [['ship', 'displacement', 'guns']]

,ship,displacement,guns
0,Kirishima,32000,8
1,South Dakota,37000,12
2,Washington,37000,12
3,California,32000,12
